In [1]:
!pip install numpy pandas matplotlib tensorflow scikit-learn


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Đọc dữ liệu từ file đã tải lên
data = pd.read_csv('../../data/BCP.csv')

# Loại bỏ cột ngày giờ (giả sử cột đầu tiên là cột ngày giờ)
features = data.iloc[:, 1:]  # Bỏ cột đầu tiên và giữ lại các cột chứa đặc trưng

# Kiểm tra dữ liệu
print(features.head())

# Chuẩn hóa các đặc trưng
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test = train_test_split(scaled_features, test_size=0.2, random_state=42)

   440FI539  440FI540  440PDI541  440PDI564  440PDIC565  440PI546  440PI551  \
0     3.823     3.316       0.01      0.375       4.649     3.635     0.463   
1     3.798     3.304       0.01      0.375       4.681     3.635     0.464   
2     3.827     3.304       0.01      0.375       4.658     3.637     0.464   
3     3.821     3.280       0.01      0.375       4.654     3.636     0.465   
4     3.830     3.278       0.01      0.376       4.683     3.636     0.464   

   440PI553  440PI558A  440PI558B  440PI558C  440PI561A  
0      0.47      1.721      1.692      1.541      1.786  
1      0.47      1.719      1.690      1.539      1.781  
2      0.47      1.721      1.692      1.540      1.783  
3      0.47      1.722      1.692      1.541      1.783  
4      0.47      1.721      1.694      1.541      1.786  


In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve

# Đọc và chuẩn bị dữ liệu
data = pd.read_csv('../../data/BCP.csv')

# Bỏ qua cột đầu tiên là ngày giờ
features = data.iloc[:, 1:]  # Giả sử cột đầu tiên là cột ngày giờ

# Chuẩn hóa các đặc trưng
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test = train_test_split(scaled_features, test_size=0.2, random_state=42)

# Kích thước của đầu vào và lớp ẩn
input_dim = X_train.shape[1]
latent_dim = 2  # Số chiều không gian tiềm ẩn

# Encoder
inputs = Input(shape=(input_dim,), dtype='float32')
h = Dense(16, activation='relu')(inputs)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

# Sampling layer
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# Decoder
decoder_h = Dense(16, activation='relu')
decoder_mean = Dense(input_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

# VAE model
vae = Model(inputs, x_decoded_mean)

# Đảm bảo x_decoded_mean có cùng dtype và shape với inputs
x_decoded_mean = tf.cast(x_decoded_mean, dtype=tf.float32)

# Tính toán reconstruction loss với kiểm tra kích thước
reconstruction_loss = tf.reduce_sum(tf.square(tf.cast(inputs, tf.float32) - x_decoded_mean), axis=-1)

# Tính toán KL divergence loss
kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)

# Tổng hợp loss
vae_loss = K.mean(reconstruction_loss + kl_loss)

vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

# Huấn luyện mô hình
vae.fit(X_train, epochs=50, batch_size=32, validation_split=0.1)

# Dự đoán và đánh giá
X_test_pred = vae.predict(X_test)
reconstruction_loss = np.mean(np.square(X_test - X_test_pred), axis=1)

# Thiết lập ngưỡng để phát hiện bất thường
threshold = np.percentile(reconstruction_loss, 95)
pred_labels = (reconstruction_loss > threshold).astype(int)

# Đánh giá mô hình
roc_auc = roc_auc_score(pred_labels, reconstruction_loss)
print(f'ROC AUC Score: {roc_auc}')

# Vẽ biểu đồ ROC
fpr, tpr, _ = roc_curve(pred_labels, reconstruction_loss)
plt.plot(fpr, tpr, marker='.')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```
